In [1]:
from lionagi.experimental.work import work, Worker

In [2]:
from lionagi import Session


class Coder(Worker):
    name = "coder"
    assignments = {"chat": "instruction, context -> response"}

    @work(assignment="instruction, context -> response", capacity=5)
    async def chat(instruction=None, context=None):
        session = Session()
        return await session.chat(instruction, context=context)

In [3]:
a = Coder()

In [4]:
# calling the function only adds it to the internal worklog

for i in range(7):
    await a.chat("Hello", context="world")

In [5]:
chat_func = a.work_functions["chat"]

In [6]:
print(len(chat_func.worklog.pile))
print(len(chat_func.worklog.pending_sequence))
print(chat_func.worklog.queue.available_capacity)

7
7
5


In [7]:
# as worklog forwards, it push the work to the queue according to the capacity

await chat_func.worklog.forward()

In [8]:
# now pending_sequence is one less than before
# and the queue available capacity reduced by 1

print(len(chat_func.worklog.pile))
print(len(chat_func.worklog.pending_sequence))
print(chat_func.worklog.queue.available_capacity)

7
2
None


In [9]:
await chat_func.worklog.queue.process()

In [10]:
completed_work = chat_func.worklog.completed_work

In [11]:
len(completed_work)

5

In [12]:
for _, v in completed_work.items():
    print(v.result)

Hello, world! How can I assist you today?
Hello, world! How can I assist you today?
Hello, world! How can I assist you today?
Hello! How can I assist you today?
Hello! How can I assist you today?


In [13]:
print(len(chat_func.worklog.pile))
print(len(chat_func.worklog.pending_sequence))
print(chat_func.worklog.queue.available_capacity)

7
2
5


In [14]:
await chat_func.worklog.forward()

In [15]:
await chat_func.worklog.queue.process()
completed_work = chat_func.worklog.completed_work
len(completed_work)

7

In [16]:
for _, v in completed_work.items():
    print(v.result)

Hello, world! How can I assist you today?
Hello, world! How can I assist you today?
Hello, world! How can I assist you today?
Hello! How can I assist you today?
Hello! How can I assist you today?
Hello! How can I assist you today?
Hello! How can I assist you today within the context of "world"?


In [17]:
print(len(chat_func.worklog.pile))
print(len(chat_func.worklog.pending_sequence))
print(chat_func.worklog.queue.available_capacity)

7
0
5
